<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/split_coding_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import datetime


plt.rc('font', family='NanumBarunGothic')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [147]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/final_ops_data_2021.csv', encoding='CP949')
df['day'] = df['day'].apply(lambda x: int(x.replace('-', '')))

df.shape

(12166, 38)

In [142]:
# 연속적 분할 함수
def order(df, period, interval, del_cri, except_print):

    first_date = df.sort_values('day').reset_index(drop=True).loc[0,'day']
    first_date = '0'+str(first_date)

    last_date = df.sort_values('day').reset_index(drop=True).loc[len(df)-1, 'day']
    last_date = '0'+str(last_date)

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()

    while True:
        
        start_date = first_date

        end_date = datetime.datetime.strptime(start_date, '%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%m%d')

        pred_date = datetime.datetime.strptime(end_date, '%m%d')
        pred_date = pred_date + datetime.timedelta(days=21)
        pred_date = pred_date.strftime('%m%d')

        if pred_date >= last_date:
            print('가져오는 데이터가 마지막 날짜 이후의 데이터입니다.')
            break

        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(df['PCODE']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
        target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

        # start_date 이후의 경기만 추출
        tr_df = df[ (df['day'] >= int(start_date)) & (df['day'] <= int(end_date)) ]
        ts_df = df[ (df['day'] > int(end_date)) & (df['day'] <= int(pred_date)) ]

        # first_date + interval
        first_date = datetime.datetime.strptime(start_date, '%m%d')
        first_date = first_date + datetime.timedelta(days=interval)
        first_date = first_date.strftime('%m%d')

        # 타자 한명씩 추출
        for player in hitter_list:

            tr_pl_data = tr_df[tr_df['PCODE'] == player]
            ts_pl_data = ts_df[ts_df['PCODE'] == player]

            away_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@') ]
            home_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@')  == False ]

            away_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@') ]
            home_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@')  == False ]

            if (len(away_tr) < del_cri) | (len(home_tr) < del_cri) | (len(away_ts) < 3.5) | (len(home_ts) < 3.5) :
                if except_print == True:
                    print('{}은/는 경기 수가 {}개보다 작기 때문에 삭제'.format(player, del_cri))
                continue

            else:

                # tr_data의 타겟 칼럼을 모두 합해서 저장
                home_sum = home_tr.loc[:, target_col].sum()
                away_sum = away_tr.loc[:, target_col].sum()
                # 어떤 선수인지 저장 (name -> pcode)
                home_sum['PCODE'] = player
                away_sum['PCODE'] = player

                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                home_ts_sum = home_ts.loc[:, target_col].sum()
                away_ts_sum = away_ts.loc[:, target_col].sum()

                home_sum['avg_label'] = home_ts_sum['hit'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['slg_label'] = home_ts_sum['tb'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['obp_label'] = (home_ts_sum['hit'] + home_ts_sum['bb'] + home_ts_sum['hpp']) / (home_ts_sum['ab'] +  home_ts_sum['bb'] + home_ts_sum['hpp'] + home_ts_sum['sf'])
                home_sum['ops_label'] = home_sum['slg_label'] + home_sum['obp_label']

                away_sum['avg_label'] = away_ts_sum['hit'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['slg_label'] = away_ts_sum['tb'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['obp_label'] = (away_ts_sum['hit'] + away_ts_sum['bb'] + away_ts_sum['hpp']) / (away_ts_sum['ab'] +  away_ts_sum['bb'] + away_ts_sum['hpp'] + away_ts_sum['sf'])
                away_sum['ops_label'] = away_sum['slg_label'] + away_sum['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                home_result = home_sum.to_frame().transpose()
                away_result = away_sum.to_frame().transpose()
                    
                # 이 선수 데이터를 전체 데이터셋에 추가
                home_split = pd.concat([home_split, home_result])
                away_split = pd.concat([away_split, away_result])
            
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
            
    return home_split, away_split

In [144]:
home_2021, away_2021 = order(df = df, period = 100, interval = 5, del_cri= 10, except_print = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


가져오는 데이터가 마지막 날짜 이후의 데이터입니다.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


In [145]:
home_2021

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,PCODE,avg_label,slg_label,obp_label,ops_label
0,106.0,17.0,25.0,4.0,0.0,3.0,38.0,19.0,1.0,0.0,17.0,1.0,0.0,11.0,3.0,0.0,3.0,468.0,0.909,76290.0,0.142857,0.357143,0.250000,0.607143
1,56.0,4.0,17.0,3.0,0.0,2.0,26.0,9.0,1.0,1.0,14.0,0.0,0.0,5.0,2.0,2.0,1.0,257.0,0.454,76802.0,0.263158,0.315789,0.263158,0.578947
2,90.0,16.0,27.0,6.0,0.0,2.0,39.0,9.0,4.0,1.0,9.0,1.0,0.0,17.0,0.0,1.0,2.0,399.0,0.399,64006.0,0.444444,0.555556,0.500000,1.055556
3,79.0,13.0,21.0,7.0,0.0,3.0,37.0,17.0,0.0,0.0,11.0,2.0,0.0,23.0,3.0,0.0,1.0,391.0,0.097,79365.0,0.300000,0.350000,0.400000,0.750000
4,16.0,10.0,5.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,5.0,0.0,0.0,6.0,0.0,0.0,0.0,100.0,0.090,64012.0,1.000000,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,103.0,12.0,28.0,2.0,1.0,0.0,32.0,8.0,1.0,1.0,11.0,0.0,1.0,13.0,1.0,3.0,2.0,407.0,0.395,64984.0,0.217391,0.304348,0.269231,0.573579
326,86.0,22.0,28.0,1.0,0.0,5.0,44.0,12.0,1.0,0.0,8.0,0.0,1.0,20.0,1.0,0.0,0.0,363.0,0.716,62947.0,0.280000,0.760000,0.321429,1.081429
327,61.0,9.0,10.0,0.0,0.0,4.0,22.0,13.0,1.0,0.0,10.0,4.0,1.0,12.0,1.0,0.0,1.0,321.0,-0.189,76267.0,0.473684,1.315789,0.565217,1.881007
328,4.0,3.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.000,69100.0,0.000000,0.000000,NaN,NaN


In [146]:
away_2021

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,PCODE,avg_label,slg_label,obp_label,ops_label
0,104.0,22.0,38.0,7.0,0.0,7.0,66.0,20.0,0.0,0.0,14.0,1.0,1.0,7.0,1.0,0.0,1.0,441.0,1.113,76290.0,0.388889,0.666667,0.421053,1.087719
1,67.0,7.0,18.0,4.0,0.0,1.0,25.0,13.0,1.0,1.0,6.0,0.0,0.0,9.0,1.0,1.0,2.0,288.0,0.050,76802.0,0.250000,0.250000,0.250000,0.500000
2,78.0,14.0,19.0,5.0,1.0,2.0,32.0,13.0,3.0,1.0,5.0,2.0,0.0,11.0,0.0,0.0,0.0,366.0,0.416,64006.0,0.111111,0.111111,0.200000,0.311111
3,98.0,18.0,29.0,0.0,0.0,13.0,68.0,29.0,0.0,0.0,15.0,2.0,0.0,28.0,1.0,0.0,0.0,461.0,1.595,79365.0,0.117647,0.352941,0.117647,0.470588
4,15.0,8.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,2.0,0.0,0.0,8.0,0.0,0.0,0.0,67.0,-0.024,64012.0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,105.0,11.0,31.0,3.0,1.0,0.0,36.0,8.0,5.0,1.0,6.0,0.0,0.0,21.0,2.0,0.0,0.0,422.0,-0.186,64984.0,0.137931,0.275862,0.137931,0.413793
326,129.0,20.0,36.0,12.0,0.0,5.0,63.0,18.0,0.0,0.0,7.0,5.0,0.0,28.0,2.0,0.0,3.0,534.0,-0.524,62947.0,0.285714,0.464286,0.333333,0.797619
327,70.0,10.0,15.0,2.0,0.0,2.0,23.0,11.0,0.0,0.0,9.0,2.0,1.0,15.0,2.0,0.0,0.0,381.0,-0.338,76267.0,0.263158,0.263158,0.363636,0.626794
328,12.0,0.0,2.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,60.0,-0.077,69100.0,0.000000,0.000000,0.000000,0.000000


# 휴지통

In [80]:
# 연속적 분할 함수
def order(df, period, interval, del_cri, except_print):

    first_date = df.sort_values('day').reset_index(drop=True).loc[0,'day']
    first_date = '0'+str(first_date)

    last_date = df.sort_values('day').reset_index(drop=True).loc[len(df)-1, 'day']
    last_date = '0'+str(last_date)

    while True:
        
        start_date = first_date

        end_date = datetime.datetime.strptime(start_date, '%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%m%d')

    pred_date = datetime.datetime.strptime(end_date, '%m%d')
    pred_date = pred_date + datetime.timedelta(days=21)
    pred_date = pred_date.strftime('%m%d')

    # 해당 시즌 타자 목록 (name -> pcode)
    hitter_list = list(set(df['PCODE']))

    # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
    target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA']

    # start_date 이후의 경기만 추출
    tr_df = df[ (df['day'] >= int(start_date)) & (df['day'] <= int(end_date)) ]
    ts_df = df[ (df['day'] > int(end_date)) & (df['day'] <= int(pred_date)) ]

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()
        
    # 타자 한명씩 추출
    for player in hitter_list:

        tr_pl_data = tr_df[tr_df['PCODE'] == player]
        ts_pl_data = ts_df[ts_df['PCODE'] == player]

        if (len(tr_pl_data) < del_cri) | (len(ts_pl_data) < del_cri) :
            if except_print == True:
                print('{}은/는 경기 수가 {}개보다 작기 때문에 삭제'.format(player, del_cri))
            continue

        else:
            away_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@') ]
            home_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@')  == False ]

            away_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@') ]
            home_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@')  == False ]
                
            # tr_data의 타겟 칼럼을 모두 합해서 저장
            home_sum = home_tr.loc[:, target_col].sum()
            away_sum = away_tr.loc[:, target_col].sum()
            # 어떤 선수인지 저장 (name -> pcode)
            home_sum['PCODE'] = player
            away_sum['PCODE'] = player

            # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
            home_ts_sum = home_ts.loc[:, target_col].sum()
            away_ts_sum = away_ts.loc[:, target_col].sum()

            home_sum['avg_label'] = home_ts_sum['hit'] / home_ts_sum['ab']
            home_sum['slg_label'] = home_ts_sum['tb'] / home_ts_sum['ab']
            home_sum['obp_label'] = (home_ts_sum['hit'] + home_ts_sum['bb'] + home_ts_sum['hpp']) / (home_ts_sum['ab'] +  home_ts_sum['bb'] + home_ts_sum['hpp'] + home_ts_sum['sf'])
            home_sum['ops_label'] = home_sum['slg_label'] + home_sum['obp_label']

            away_sum['avg_label'] = away_ts_sum['hit'] / away_ts_sum['ab']
            away_sum['slg_label'] = away_ts_sum['tb'] / away_ts_sum['ab']
            away_sum['obp_label'] = (away_ts_sum['hit'] + away_ts_sum['bb'] + away_ts_sum['hpp']) / (away_ts_sum['ab'] +  away_ts_sum['bb'] + away_ts_sum['hpp'] + away_ts_sum['sf'])
            away_sum['ops_label'] = away_sum['slg_label'] + away_sum['obp_label']

            # Series 데이터를 DataFrame 형식으로 변환
            home_result = home_sum.to_frame().transpose()
            away_result = away_sum.to_frame().transpose()
                
            # 이 선수 데이터를 전체 데이터셋에 추가
            home_split = pd.concat([home_split, home_result])
            away_split = pd.concat([away_split, away_result])
        
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
        
    return home_split, away_split